# Introduction to Numerical Modelling for Water Systems

In this lab you will write a simple model for how water infiltrates a porous soil column to replenish soil water storage. These exercises are intentionally simple, aiming to build up your conceptual understanding of how mathematical models can be developed and applied to environmental systems. The principles and concepts you will learn underlie most numerical models of water systems and are thus broadly applicable, rather than specific to the idealised system you will simulate in this lab.

The aims of this lab are to:
* Understand how a numerical model is constructed 
* Implement mathematical equations to describe how stored water volume changes through time
* Understand the importance of initial and boundary conditions in mathematical modelling
* Create and explain graphs/figures that visualise model behaviour
* Experiment with the model to understand its sensitivity to different parameters
* Understand the concepts of transient evolution and steady-state dynamic equilibrium
* Explore the concepts of model calibration and verification

### Python
The model is written in the Python computer programming language. But don't worry, you won't need any prior knowledge of computer programming. We have set up this lab so it will also serve as a basic introduction to programming. 

Learning to program is not the main aim but we want you to see what's "under the hood" of simple models to help you understand how they are built and what they can and cannot do! However, scientific computing is becoming more common place in research and consultancy, so it won't do you any harm to see it in action. Python is highly versatile, for example it can interface with GIS to automate workflows, or it is used in many data analysis and machine learning applications.

**To run a code cell, click in a cell, hold down shift, and press enter.** An asterisk in square brackets `In [*]:` will appear while the code is being executed, and this will change to a number `In [1]:` when the code is finished. *The order in which you execute code cells matters, they must be run in sequence. If the code fails, try restarting and executing all cells again from the top.* To restart the script click the circular arrow button or select `Kernel / Restart` from the menu bar at the top of the window.

Inside blocks of python code each line is a command which will be executed in sequence from top to bottom. Once a variable is created in a code cell it remains active and can continue to be used in further code cells below. There are comments indicated by lines that start with `#`. These lines are not computer code but rather provide commentary and information about what the code is doing to help you follow along. 

**Good commenting is one of the most important aspects of computer programming!** If you modify code or write some from scratch, always document what each line of code is meant to do with a concise comment!

Python has many tools and we don't need to use all of them (it would take forever to load) so we have to tell python which modules we want to use:

In [ ]:
# import some modules that we will need for numerical calculations and for plotting
# we'll shorten their names using "as" so that we don't need to type much later on

# import modules for numerical calculations and for plotting
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

# tell python to allow plotting to occur within the page
%matplotlib inline

# Customise figure style to use font size 16
from matplotlib import rcParams
rcParams['font.size'] = 16

# import colormaps so we can use shaded values as a function of time
from matplotlib import cm

<img style="float: right; width: 120px; margin: 10px" src="images/1Dsketch.png" alt="Drawing" />

## 2.0 1-D column model

Today we will implement a numerical model solving the Richards Equation (Richards, 1931) for soil water infiltration down a 1-D vertical column. The Richards equation is a so-called partial differential equation (PDE, has derivatives of both time and space) and is based on Darcy's Law. It represents the vertical segregation of water into unsaturated porous soil. Units are expressed as $[L]$ength and $[T]$ime.

Our starting point is a soil column conceptualised as a vertical 1-D profile into a laterally homogeneous soil horizon from the surface down to bedrock. To discretise the model we split up the soil column into discrete volume elements (like a stack of bathtubs). The model will track the water content in each of these volume cells and compute the flux of water that flows vertically across the faces between them. We will assume that the only input of water is precipition that infiltrates into the soil from above, and the only output is recharge of ground water from water passing out the base and into a bedrock aquifer. Precipitation and recharge are the boundary conditions we will impose on the top and base of our 1-D model domain. We assume there is no lateral flow of water in or out of the soil column, representing a laterally homogeneous layer.

Rather than tracking the absolute volume of water, $V_\mathrm{water}$, in each volume element, we will consider the relative volume fraction, $\theta = V_\mathrm{water}/V_\mathrm{total}$. Based on the fundamental principle of mass conservation, we can express the governing equation for the evolving water content with depth and time as,

\begin{equation}
    {{\partial \theta(z,t)}\over{\partial t}}=-{{\partial q(z,t)}\over{\partial z}} \ , \tag{1} \label{eq:1}
\end{equation}

where the change in volume fraction of water in each volume element $\theta(t,z)$ $[L^3/L^3 = 1]$ through time $t$ $[T]$ is given by the vertical gradient in volumetric water flux $q(z,t)$ $[L^3/L^2/T = L/T]$ with depth $z$ $[L]$. 

Equation \eqref{eq:1} states that if there is a change in the flux of water with depth, $dq/dz$, this has to be balanced by a change in soil water content $\theta$. If more water arrives in the top of a volume element than leaves out the base, the stored amount in the local volume will increase, and vice versa.

For saturated soil, Darcy's law can be used to describe the water flux $q$ $[L/T]$ as the product of the hydraulic conductivity $K$ $[L/T]$ and the vertical gradient of the hydraulic head  $dH / dz$ $[L/L = 1]$,

\begin{equation}
    q = -K{{\partial H}\over{\partial z}} \ . \tag{2} \label{eq:2}
\end{equation}

In unsaturated soil, however, the flow of water is governed by two pressure heads (i.e., mechanical driving forces). One pressure component is due to the buoyancy difference between water and air that occupies the unsaturated pore space, $\Delta \rho g z$, with $\Delta \rho = \rho^\mathrm{water} - \rho^\mathrm{air}$ the density difference, and $g$ the gravity. The buoyancy-related pressure is transformed to a hydraulic head through normalising by the buoyancy force $\Delta \rho g$, leaving just $z$.

A second pressure component driving flow is due to capillary forces in the unsaturated soil acting to effectively suck water into air-filled pores to wet the surrounding soil particles. The theoretical derivation of that capillary pressure component is somewhat involved; hence, we will represent it with an empirically determined function $h$ (units are $[L]$, as it is also normalised by $\Delta \rho g$). 

The total hydraulic pressure head then is a combination of capillary and buoyancy-related heads, $H=h-z$; note that the negative sign comes from the fact that we have defined the z-axis to point down from the surface into the soil column. With that, we can rewrite equation \eqref{eq:2} as,

\begin{equation}
    q=-K{{\partial(h+z)}\over{\partial z}} = -K\left({{\partial h}\over{\partial z}}-{{\partial z}\over{\partial z}}\right) \ . \tag{3} \label{eq:3}
\end{equation}

Since ${\partial z}/{\partial z} = 1$, we can simplify to:

\begin{equation}
    q=-K\left({{\partial h}\over{\partial z}}-1\right) \ . \tag{4} \label{eq:4}
\end{equation}

Substituting eq. (4) into the conservation equation (1), and noting that both the hydraulic conductivity $K$ and the capillary head $h$ are themselves empirical functions of the soil water content $\theta$, we finally get to the Richards equation,

\begin{equation}
    {{\partial \theta}\over{\partial t}}=-{{\partial}\over{\partial z}} \left[-K(\theta) \left( {{\partial h(\theta)}\over{\partial z}}-1 \right) \right] \ . \tag{5} \label{eq:5}
\end{equation}

So, how does the capillary pressure head vary as a function of water content? We would expect capillary suction to vanish as the soil becomes water-saturated, and highest if the soil is nearly dry. An expression for the relationship between water content and pressure due to capillary forces $h$ can be found from rearranging eq. (3) in *van Genuchten* (1980) as,

\begin{equation}
h(\Theta) = -{{1}\over{\alpha}} \left(\dfrac{1}{\Theta^{1/m}}-1 \right)^{1/n} \ . \tag{6} \label{eq:6}
\end{equation}

Next, we need an expression that relates the hydraulic conductivity to the water content. Hydraulic conductivity is greatest in saturated soils and vanishes in dry soils; *van Genuchten* (1980) has it tailing off as a power law,

\begin{equation}
    K(\Theta) = K_s \Theta^{1/2}\:\left[1-(1-\Theta^{1/m})^m\right]^2 \ , \tag{7} \label{eq:7}
\end{equation}

where $K_s$ is the conductivity in saturated soil.

In eqs (6) and (7), $\Theta$ is the relative water content, varying between 0, corresponding to the residual water content, $\theta_r$, and 1, corresponding to the saturated water content, $\theta_s$, according to,

\begin{equation}
    \Theta ={{\theta-\theta_r}\over{\theta_s-\theta_r}} \ . \tag{8} \label{eq:8}
\end{equation}


<img style="float: right; width: 300px; margin: 20px" src="images/DiscreteStencil.png" alt="Drawing" />

## 2.1 Numerical Implementation

To calculate a numerical solution we now need to discretise the governing equation (1). We discretise the vertical coordinate into $n$ volume elements $z^i$, $i=1,2,...,n$, spaced at constant grid size, $\Delta z$. Time we discretise into $m$ time steps $t^k$, $k=1,2,...,m$, spaced at constant step size $\Delta t$. Using the finite-difference method, we write the numerical governing equation as,

\begin{equation}
    {{\theta^{i,k} - \theta^{i,k-1}}\over{\Delta t}}= - {{q^{i+1/2,k-1} - q^{i-1/2,k-1}}\over{\Delta z}} \ , \tag{9} \label{eq:9}
\end{equation}

where the discrete flux values into the top ($q^{i-1/2}$) and out the bottom ($q^{i+1/2}$) of each volume element are given by the relations,

\begin{align}
q^{i+1/2} = - \dfrac{K^{i+1}+K^{i}}{2} \left(\dfrac{h^{i+1}-h^{i}}{\Delta z} - 1 \right) \ , \tag{10} \label{eq:10} \\
q^{i-1/2} = - \dfrac{K^{i-1}+K^{i}}{2} \left(\dfrac{h^{i}-h^{i-1}}{\Delta z} - 1 \right) \ . \tag{11} \label{eq:11} \\
\end{align}

The figure to the right shows the so-called discretisation stencil, which visualises the spatial relationships and indexing of adjacent discrete cells, and fluxes between them, that are needed to write the discrete governing equation in the cell $i$. You can see that the half indices $i-1/2$ and $i+1/2$ refer to quantities located on the top and base faces of the cell $i$, respectively. It is convention to number grid cells in the direction of the coordinate axis. Here, our vertical coordinate axis, $z$, points downwards from the surface into the soil, and therefore the index numbering goes into the downwards direction as well.

**When using the finite-difference method, we always take the differences from the higher to the lower index.** Always pay close attention to the indexing in your model code, it is one of the most frequent sources of annoying little errors or *bugs* in numerical modelling.

## 2.2 Model Calibration

Before programming the numerical algorithm, let's first examine the empirical relationships between capillary head, $h$, hydraulic conductivity, $K$, and the relative saturation, $\Theta$. The model calibration assumes that water content can vary between a residual content, $\theta_r$, and a saturated state $\theta_s$. We'll use typical values of saturated hydraulic conductivity for silt loam soil and use empirical parameters for the *van Genuchten* (1980) relationships taken from that paper. Note that units are internally consistent, but they are not SI base units!

In [ ]:
# take soil properties for a silt loam (van Genuchten 1980)
# theta is relative water content cm3/cm3 volume per total volume of soil
# i.e. at saturated porosity theta = 1 and wilting point theta = 0!

theta_s = 0.396           # [cm3/cm3] saturated
theta_r = 0.131           # [cm3/cm3] residual
Ks      = 4.96            # [cm/day] saturated conductivity
alpha   = 4.23*10.**(-3.) # [1/cm] prefactor
n       = 2.06            # [1] powerlaw coefficient
m       = 1.-1./n         # [1] powerlaw coefficient

For later convenience, we define functions that we can call every time we wish to calculate $\Theta$, $K$, and $h$ according to the equations above.

___INSTRUCTION: Use the formula from eq. (8) to fill in the blank in the first function below to calculate the relative saturation from the water content.___

In [ ]:
# define a function to convert absolute water content to relative saturation (scaling between 0 and 1)
def get_THETA(theta):
    Theta =   # <== use the formula from eq. (8) to complete the calculation of relative saturation here
    return Theta

# define a function to calculate the hydraulic conductivity as a function of relative saturation
def get_K(theta):
    Theta = get_THETA(theta)
    K = (Theta**0.5)*(1-(1-Theta**(1./m))**m)**2.
    return K

# define a function to calculate the capillary head as a function of relative saturation
def get_h(theta):
    Theta = get_THETA(theta)
    h = -(1./alpha)*(1/Theta**(1/m) - 1.)**(1./n)
    return h

Now let's plot the relationships between soil water content, hydraulic conductivity, and capillary pressure head to get a better understanding of the relationships bewteen them.

___QUESTION: What features of these calibrated parameter curves do you notice, and what do you anticipate is their  effect on the physical model outcome?___

In [ ]:
# define the full range of theta values between residual content and saturation point
theta = np.linspace(theta_r,theta_s,500)[1:-1]

# call the functions for capillary head and hydraulic conductivity
h = get_h(theta)
K = get_K(theta)

# plot the capillary head
plt.figure(1)
ax = plt.subplot(111)
colour = [0.5,0.5,0.8]
plt.plot(theta, -h, '-',color=colour)
plt.yscale('symlog') # to make y-axis log scale
plt.xlabel('water content [1]')
plt.ylabel('capillary head (–1) [cm]',color=colour)

# colour the left axis
ax.spines['left'].set_color(colour)
ax.spines['top'].set_color(colour)
ax.yaxis.label.set_color(colour)
ax.tick_params(axis='y', colors=colour)

# plot the hydraulic conductivity
plt.twinx()
plt.plot(theta, K, 'k-')
plt.ylabel('rel. hydraulic conductivity [1]');

<div class="alert alert-block alert-info">
<font color="black">
<h3>Task 1</h3>
<p><p> Since both relations vary as a function of water content, make a plot to explore how the pressure head varies with the hydraulic conductivity in the empty code block below.
<p></p>
</font>
</div>

In [ ]:
# create a plot of hydraulic conductivity over pressure head
# <== use code cells above to get you started




## 2.3 Numerical Model Setup

Now that we have all we need to program the numerical algorithm to solve the Richards Equation, it's time to prepare the model setup. We will define some parameters that will define our spatial domain size.

Note that because of the empirical relations for $K$ and $h$, our model time is going to be in units of [days], and our model space in [cm]. 

___QUESTION: Can you figure out what exactly the function `np.arange()` does? Look it up online if necessary.___

In [ ]:
# The model domain goes from zero at the surface to a given depth, divided into grid steps of set length
depth = 100.     # domain depth [cm]
dz    = 1.       # grid spacing [cm]
z     = np.arange(0.    ,depth      ,dz)  # coordinate vector for grid cell centre nodes
zm    = np.arange(-dz/2.,depth+dz/2.,dz)  # coordinate vector for grid cell face nodes

# The model will start at time zero days and run to a set stopping time, divided into steps of set length
t    = 0.     # starting time [days]
dt   = 0.0001 # discrete time step [days]
tend = 2.     # stopping time [days]

# Set up controls for when to write progress and plot results
PlotTime  = 0.      # plot initial condition
dPlotTime = 5./24.  # plot every few hours

The next code block creates a function that initialises an empty figure with the desired format that we will use to plot the model output on as the model run proceeds. Don't worry too much about the detail here but do quickly take a look through to see if you can understand how it's done.

In [ ]:
def init_fig(z,theta_r,theta_s,Pe):

    # Set up the figure and add the axis
    fig = plt.figure(2,figsize=(12,8))
    ax1 = fig.add_subplot(121)

    # Set the x axis limits, move labels to the top of the box and label
    ax1.set_xlim(0.08,0.45)
    ax1.xaxis.tick_top()
    ax1.set_xlabel("Soil Moisture Content [vol]")
    ax1.xaxis.set_label_position('top') 

    # Set the y axis limits and invert the y axis
    ax1.set_ylim((np.min(z), np.max(z)))
    ax1.invert_yaxis()
    ax1.set_ylabel("Depth [cm]")

    #plot min and max soil moisture content
    ax1.plot([theta_r,theta_r],[0,100],'k--',lw=0.5)
    ax1.text(theta_r-0.03,60,'Residual soil moisture $\Theta_r$',rotation=90)
    ax1.plot([theta_s,theta_s],[0,100],'k--',lw=0.5)
    ax1.text(theta_s+0.015,60,'Saturated soil moisture $\Theta_s$',rotation=90)

    # Create an empty line object that we will populate repeatedly to create the animation
    theta = np.zeros(len(z)) + theta_r + 1.e-6
    ax1.plot(z,theta,'k-');

    # Set up the figure and add the axes
    ax2  = fig.add_subplot(122)

    # Set the x axis limits, move labels to the top of the box and label
    ax2.set_xlim(0,Pe+Pe/6)
    ax2.xaxis.tick_top()
    ax2.set_xlabel("Water segregation flux [cm/day]")
    ax2.xaxis.set_label_position('top') 

    # Set the y axis limits and invert the y axis
    ax2.set_ylim((np.min(z), np.max(z)))
    ax2.invert_yaxis()

    #plot min and max soil moisture content
    ax2.plot([Pe,Pe],[0,100],'k--',lw=0.5)
    ax2.text(Pe+0.25,60,'Precipitation flux Pe [cm/day]',rotation=90)

    # Create an empty line object that we will populate repeatedly to create the animation
    q = np.zeros(len(zm))
    ax2.plot(zm,q,'k-');

    return fig, ax1, ax2

For the **inflow boundary condition** at the top of the domain, we will set an effective precipitation rate, $P_e$ [cm/day], at which water is being delivered to the soil for infiltration (remember, effective precipitation = precipitation – evapotranspiration). For the **outflow boundary condition** at the base of the column we will be setting a free-flow condition, where the outflux from the base of the column is the same as the dynamically evolving influx into the lowermost grid cell $(q^\mathrm{out} = q^{i=n-1/2})$. That way, however much water makes its way down the full vertical column will be allowed to drain into the aquifer below with no further hydraulic resistance.

We'll set the initial condition for $\theta$ to be very slightly above the residual water content ($\theta^{i,0} = \theta_r + \epsilon$) throughout the soil column so that the soil is effectively bone dry to start with. Note that we need to track theta at each point in the soil column, so we will make it the same length as the depth coordinate vector as `z`.

Finally we need to set timing parameters that control how long the model will run for, what size of time step will be taken, and how often the model will plot output to the prepared figure.

<div class="alert alert-block alert-info">
<font color="black">
<h3>Task 2</h3>
<p> In the model code below, locate the line where the soil water content, $\theta$, is updated and fill in the formula you find when reordering the discretised model eq. (9) above into the form of an explicit update. Use the numpy function np.diff() to take the finite difference of the flux $q$.

Hint 1: An explicit update is when a single unknown quantity appears to the left, and all known quantities to the right of the equal sign.
    
Hint 2: Check how the flux q is calculated from the finite difference of the hydraulic head in the provided code below. You should be able to follow the same pattern to calculate the update to the water content.
    
Once you've completed the code, execute the cell to run the model and see if you can understand what the results show.
<p></p>
</font>
</div>

In [ ]:
################################
#####  MAIN MODEL ROUTINE  #####
################################

# set the top boundary condition
Pe = 5. # preciptitation rate in [cm/day]

# set initial condition for theta to be every so slightly above the residual water content
theta = np.zeros(len(z)) + theta_r + 1.e-6 + 1e-3*(depth-z)/depth

# initialise a vector to calculate flux values on cell faces
q     = np.zeros(len(z)+1)

# reset timing parameters
t         = 0.     # starting time [days]
dt        = 1e-4   # discrete time step [days]
tend      = 2.     # stopping time [days]
PlotTime  = 0.     # initialise plot time count
dPlotTime = 5./24. # plotting time interval (every 5 hours)

# initialise figure to plot model results
fig_out, ax1, ax2 = init_fig(z,theta_r,theta_s,Pe);

while t < tend:
 
    # get capillary and total pressure head
    h = get_h(theta)
        
    # get hydraulic conductivity
    K = Ks * get_K(theta)

    # interpolate conductivity values to cell boundaries
    Km = (K[1:] + K[:-1])/2

    # get water flux on cell boundaries
    q[1:-1] = - Km * (np.diff(h)/dz - 1)
    
    # limit q to remain between zero and top influx value
    q  = np.clip(q,0+1e-6,Pe-1e-6)
    
    # set constant in/out-flux boundary conditions on top and base of domain
    q[ 0] = Pe;     # influx boundary is precipitation
    q[-1] = q[-2];  # outflux boundary free outflux (dq/dz = 0)
    
    # update theta with the flux gradient
    theta -=   # <== fill in the formula to update water content according to the explicit form of eq. (9)
               #     use the function np.diff() to take the finite difference for the flux gradient
    
    
    # limit theta to remain between residual and saturation points
    theta  = np.clip(theta,theta_r+1e-6,theta_s-1e-6)
    
    # plot the results
    if t > PlotTime:
        print('    ---  dt = %4.4e;  t = %4.4e;  max q = %4.4e;  max theta = %4.4e;' % (dt,t,np.max(q[1:-1]),np.max(theta)))
        ax1.plot(theta,z ,color=cm.copper(t/tend),label=str(t))
        ax2.plot(q,zm,color=cm.copper(t/tend),label=str(t))
        PlotTime += dPlotTime
        
    # update Time
    t += dt

## 2.4 Numerical Stability

You should now be able to see from the model output that there's indeed a wetting front propagating down into the solid column. So far so good, the model seems to run! Let's see what happens if we run the same model again but for slightly longer.

<div class="alert alert-block alert-info">
<font color="black">
<h3>Task 3</h3>
<p>Return to the previous cell, adapt the stopping time to higher value and let the model run again! What do you observe? What happens when you change the time step to slightly higher or lower values?
<p></p>
</font>
</div>

You should have found out by now that the model is actually very unstable! If the time step is too large, the model goes into an unstable mode where both the solution for water content and the calculated fluxes between cells begin to oscillate wildly. If you wanted to run the model stably until the wetting front reaches the bedrock you need a time step as small as $10^{-5}$ days, or less than a second!

This phenomenon is called **numerical instability**. There is a whole field of research to theoretically determine what discretisation produces stable results for all kinds of physical model equations. If you go and look up "Richards Equation stability" online you will find a whole host of research papers dedicated to the question of how to avoid unstable solutions. You find the PDF for one recent review paper on the subject in the folder for this Lab (*Farthing* et al., 2017).

To be very brief, what triggers the type of instability we see here is when the time-stepping algorithm takes steps that cause the soil water content to advance downwards at a rate of more than half a grid cell per time step. If that happens, the algorithm "overshoots" the correct solution, which then causes the next time step to "undershoot" in turn, and so on.

We can fix the issue by thinking about what process sets the rate at which the water content is migrating down the domain. It seems intuitive to think that water is simply transported along by the flux $q$, like a ship that is swept along with the current of a river. If that's the case, then the stable time step size would be defined by the so-called [Courant-Friedrichs-Lewy (CFL) condition](https://en.wikipedia.org/wiki/Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition),

\begin{equation}
    \Delta t_\mathrm{stable} \leq \dfrac{1}{2} \dfrac{\Delta z}{\max \left(|q| \right)} \ , \tag{12} \label{eq:12}
\end{equation}

which makes sure the time step is not larger than it takes to cover half a grid size going at the speed of $q$.

<div class="alert alert-block alert-info">
<font color="black">
<h3>Task 4</h3>
<p>Test if the CFL condition results in a stable numerical performance of the model! Copy the model routine from above into the empty cell below. Find the line in the model algorithm where the updated flux values are calculated. On the very next line, enter the statement provided at the top of the cell below to calculate the stable time step every time the flux calculation is updated.
<p></p>
</font>
</div>

In [ ]:
# copy down the model code above and insert this line of code at the appropriate place
# to limit the time step according to the CFL condition

dt = dz/2/np.max(abs(q))  # CFL condition




___And, did it work?___

No it didn't work, did it? In fact, you can probe the model and find that the time step needs to be more than 1000 times smaller than the CFL condition to give a stable result? Why is that?

If we take a closer look at Richards Equation, we realise that $\theta$ isn't just like a boat floating down the imaginary river of $q$. Instead, the flux is itself a non-linear function of the water content, both through the hydraulic conductivity and the capillary head! This non-linearity means that the speed of $q$ doesn't quite limit the rate at which water migrates into the soil column. 

Here's the solution to the conundrum: If we substitute $\partial h/\partial \theta \times \partial \theta/\partial z$ for the spatial derivative $\partial h(\theta)/\partial z$ in equation (5) (chain rule!) we can express the evolution of water content in terms of its own vertical gradient of water content,

\begin{equation}
{{\partial \theta}\over{\partial t}}= -{{\partial}\over{\partial z}} \left( -D \dfrac{\partial \theta}{\partial z} + K \right) \ , \tag{13} \label{eq:13}
\end{equation}

where we have introduced the hydraulic diffusivity, $D = K(\theta) \times {dh(\theta)}/{d\theta}$; $D$ has units of $[L^2/t]$. 

What this operation reveals is that Richards Equations is in fact a non-linear diffusion equation. An intuitive way to understand how a diffusion process works is to consider the example of heat conduction. For example, if you hold the tip of a cold iron bar into a hot flame it will slowly warm all the way through until it begins to burn your hand. The heat is moving along the iron without the iron itself moving along the bar: that's diffusion.

The same is true for how capillary forces act on water content. The water diffuses into the soil, meaning it moves with a different effective rate than suggested by evaluating the flux (like the heat moves along the iron without the iron itself moving at all). Luckily, there is a well known stable time step criterion for diffusive transport, and it depends on the diffusivity, $D$:

\begin{equation}
    \Delta t_\mathrm{stable} \leq \dfrac{1}{4} \dfrac{\Delta z^2}{\max \left(D \right)} \ . \tag{14} \label{eq:14}
\end{equation}

To use eq. (13) to construct a stable version of our numerical model, we need to calculate the derivative of the capillary head with respect to the water content. We can find that by taking the derivative of eq. (6), but to do that manually is a bit of a hassle. Instead, we can also use Python for this purpose. Python has a module called `sympy` that allows to do symbolic math. Here's how it works:

In [ ]:
# to use symbolic maths, we first define a few symbols for our variables and parameters
# note: use different names than above to not overwrite our numerical variables and parameters
a  = sp.symbols('alpha')
x  = sp.symbols('theta')
X  = sp.symbols('Theta')
x0 = sp.symbols('theta_r')
x1 = sp.symbols('theta_s')
m1 = sp.symbols('m')
n1 = sp.symbols('n')

# define how Theta (X) depends on theta (x)
X = (x-x0)/(x1-x0)

# define a symbolic function of how the capillary head depends on Theta
f = -(1./a)*(1/X**(1./m1) - 1.)**(1./n1)

# get the symbolic derivative of that function
dfdx = sp.diff(f,x)

# substitute 'Theta' back for 'X' and print result
print("dh_dth =",dfdx.subs(X,'Theta'))

<div class="alert alert-block alert-info">
<font color="black">
<h3>Task 5</h3>
<p> Test if the diffusion time step limiter in eq. (14) results in a stable numerical performance of the model! 
    
In the box below you find a template for a function to calculate the hydraulic diffusivity according to the symbolic result in the cell above. 
    
Complete the function to get $D$ and use it in the cell after to finally build a stable numerical model of Richards Equation.
    
Hint: you will need to use the numpy function np.diff() again to complete the flux calculation in the model code below.
<p></p>
</font>
</div>

In [ ]:
# define a function to calculate the hydraulic diffusivity
def get_D(theta):
    Theta  = get_THETA(theta)
    K      = get_K(theta)
    dh_dth =   # <== fill in result of symbolic differentiation in cell above
    D      =   # <== fill in formula for D listed below eq. (13)
    return D

In [ ]:
# set calibration parameters
theta_s = 0.396           # [cm3/cm3] saturated
theta_r = 0.131           # [cm3/cm3] residual
Ks      = 4.96            # [cm/day] saturated conductivity
alpha   = 4.23*10.**(-3.) # [1/cm] prefactor
n       = 2.06            # [1] powerlaw coefficient
m       = 1.-1./n         # [1] poweraw coefficient

# set model timing parameters
t    = 0.      # starting time [days]
tend = 4.      # stopping time [days]

# set plotting parameters
PlotTime  = 0.      # plot initial condition
dPlotTime = 5./24.  # plot every few hours

# set model domain parameters
depth = 100.     # domain depth [cm]
dz    = 1.       # grid spacing [cm]
z     = np.arange(0.    ,depth      ,dz)  # coordinate vector for grid cell nodes
zm    = np.arange(-dz/2.,depth+dz/2.,dz)  # coordinate vector for grid cell faces

# set the top boundary condition
Pe = 5. # preciptitation rate in [cm/day]

# set initial condition for theta to be every so slightly above the residual water content
theta = np.zeros(len(z)) + theta_r + 1.e-6 + 1e-3*(depth-z)/depth

# initialise a vector to calculate flux values on cell faces
q     = np.zeros(len(z)+1)

# initialise figure to plot model results
fig_out, ax1, ax2 = init_fig(z,theta_r,theta_s,Pe);

while t < tend:

    # get hydraulic conductivity
    K  = Ks * get_K(theta)
    Km = (K[1:] + K[:-1])/2
    
    # get hydraulic diffusivity
    D  = Ks * get_D(theta)
    Dm = (D[1:] + D[:-1])/2

    # get water flux on cell boundaries
    q[1:-1] =   # <== fill in flux formula from eq. (13)
    
    # limit q to remain between zero and top influx value
    q  = np.clip(q,0+1e-6,Pe-1e-6)
    
    # set constant in/out-flux boundary conditions on top and base of domain
    q[ 0] = Pe;     # influx boundary is precipitation
    q[-1] = q[-2];  # outflux boundary free outflux (dq/dz = 0)
    
    # limit stable time step for diffusive and advective transport
    dt = np.min([dz**2/4/np.max(Dm), dz/8/np.max(abs(q))])

    # update theta with the flux gradient
    theta -= np.diff(q)/dz * dt
    
    # limit theta to remain between residual and saturation points
    theta  = np.clip(theta,theta_r+1e-6,theta_s-1e-6)
    
    # plot the results
    if t > PlotTime:
        print('    ---  dt = %4.4e;  t = %4.4e;  max q = %4.4e;  max theta = %4.4e;' % (dt,t,np.max(q[1:-1]),np.max(theta)))
        ax1.plot(theta,z ,color=cm.copper(t/tend),label=str(t))
        ax2.plot(q,zm,color=cm.copper(t/tend),label=str(t))
        PlotTime += dPlotTime
        
    # update Time
    t += dt

<div class="alert alert-block alert-info">
<font color="black">
<h3>Task 6</h3>
<p>With the stable model implementation, try modifying the model parameters and boundary conditions and explore how this influences model behaviour. Keep track of the parameters you change by keeping notes and compiling a table of simulations. Be sure to change the filename when you save output figures (execute cell below to save) so that you can compare back to previous runs. It is best practice to only vary one parameter at a time to isolate its effects, and to systematically test a range of parameters at reasonable intervals. Have fun!
<p></p>
</font>
</div>

In [ ]:
# save the results as an image file
# rename this for every parameter variation you test
fig_out.savefig("Richards_Eq_demo.png",dpi=300)